In [4]:
## 37.8 
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [5]:
import pandas as pd

In [6]:
ratings_movies = pd.read_csv('data/ratings_movies.csv', sep=',')

In [7]:
#У скольких фильмов не указан год их выпуска?
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [15]:
#Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
mask = ratings_movies['year_release'] == 1999 # фильтрация по условию «год выпуска».
#группировка по названию фильма(title) и посчитаем среднее по столбцу rating для каждого фильма
#отсортируем результат в порядке возрастания.
ratings_movies[mask].groupby('title')['rating'].mean().sort_values() 


title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [16]:
# Какое сочетание жанров фильмов, выпущенных в 2010 году, получило наименьшую среднюю оценку?
mask2 = ratings_movies['year_release'] == 2010
ratings_movies[mask2].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [17]:
#Какой пользователь посмотрел фильмы наибольшего количества жанров?
ratings_movies.groupby('userId')['genres'].nunique().sort_values()

userId
214     13
85      13
245     13
494     15
578     15
      ... 
474    395
380    399
448    403
414    482
599    524
Name: genres, Length: 610, dtype: int64

In [18]:
#Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
#Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# используем метод agg() на сгруппированных данных.
# Сгруппируем таблицу по пользователям (userId).
# Для каждого пользователя рассчитаем количество выставленных им оценок и средний рейтинг.
# Отсортируем таблицу по возрастанию количества оценок и по убыванию среднего рейтинга.
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [24]:
# 37.8.6 Найдите сочетание жанров фильма, выпущенного в 2018 году, который имеет наибольший средний рейтинг, 
# и при этом число его просмотров больше 10.
mov_year_filt = ratings_movies['year_release']==2018
grouped = ratings_movies[mov_year_filt].groupby('genres')['rating'].agg(
    ['mean','count']
)
grouped[grouped['count']>10].sort_values(
    by=['mean','count'],
    ascending=[False, False]
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


## Задание 37.8.7
Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы.

In [29]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year


Составьте сводную таблицу, индексами которой являются годы выхода фильмов (year_rating), столбцами — жанры (genres), а в качестве значений выступают средние оценки фильмов.

In [52]:
pivot = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    aggfunc='median',
    fill_value=0
)
display(pivot['Animation|Children|Mystery'])

year_rating
1996    0
1997    0
1998    0
1999    0
2000    0
2001    0
2002    0
2003    0
2004    0
2005    0
2006    0
2007    0
2008    0
2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    5
Name: Animation|Children|Mystery, dtype: int64